In [ ]:
import theano
import numpy as np
import theano.tensor as T
import ch9util
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
import dautil as dl
from IPython.display import HTML
import warnings

warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
temp = dl.data.Weather.load()['TEMP'].dropna()
X = temp.values[:-1]
y = temp.values[1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=16)
w = theano.shared(0., name ='w')
c = theano.shared(0., name ='c')

In [ ]:
x = T.vector('x')
y = T.vector('y')

prediction = T.dot(x, w) + c
cost = T.sum(T.pow(prediction - y, 2))/(2 * X_train.shape[0])

gw = T.grad(cost, w)
gc = T.grad(cost, c)

learning_rate = 0.01
training_steps = 10000

In [ ]:
train = theano.function([x, y], cost, updates =
                        [(w, w - learning_rate * gw),
                         (c, c - learning_rate * gc)])
predict = theano.function([x], prediction)

In [ ]:
%matplotlib inline
for i in range(training_steps):
    train(X_train.astype(np.float), y_train)


preds = predict(X_test)
ch9util.npy_save('theano', preds)
r2 = r2_score(preds, y_test)
HTML(ch9util.scatter_predictions(preds, y_test, '', r2))